# Sistema de control de existencias
## Detector de frutas en las imágenes
En este apartado se detectan las piezas de fruta con el modelo entrenado y se apuntan los resultados en un csv por imagen. Generando un histórico de piezas de fruta por imagen y dia.

In [ ]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile

from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image

# This is needed since the notebook is stored in the object_detection folder.
sys.path.append("..")
from object_detection.utils import ops as utils_ops
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util

# This is needed to display the images.
%matplotlib inline

In [ ]:
# Path to frozen detection graph. This is the actual model that is used for the object detection.
PATH_TO_FROZEN_GRAPH = 'mask_rcnn_inception_resnet_v2_atrous/frozen_inference_graph.pb'

# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = 'label_map.pbtxt'

#Number of classes
NUM_CLASSES = 6

In [ ]:
detection_graph = tf.Graph()
with detection_graph.as_default():
  od_graph_def = tf.GraphDef()
  with tf.gfile.GFile(PATH_TO_FROZEN_GRAPH, 'rb') as fid:
    serialized_graph = fid.read()
    od_graph_def.ParseFromString(serialized_graph)
    tf.import_graph_def(od_graph_def, name='')

In [ ]:
label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
category_index = label_map_util.create_category_index(categories)

In [ ]:
def load_image_into_numpy_array(image):
  (im_width, im_height) = image.size
  return np.array(image.getdata()).reshape(
      (im_height, im_width, 3)).astype(np.uint8)

In [ ]:
images=os.listdir('drive/My Drive/Test')
TEST_IMAGE_PATHS = [ os.path.join('drive/My Drive/Test', '{}'.format(i)) for i in images ]
# Size, in inches, of the output images.
IMAGE_SIZE = (12, 8)

In [ ]:
def run_inference_for_single_image(image, graph):
  with graph.as_default():
    with tf.Session() as sess:
      # Get handles to input and output tensors
      ops = tf.get_default_graph().get_operations()
      all_tensor_names = {output.name for op in ops for output in op.outputs}
      tensor_dict = {}
      for key in [
          'num_detections', 'detection_boxes', 'detection_scores',
          'detection_classes', 'detection_masks'
      ]:
        tensor_name = key + ':0'
        if tensor_name in all_tensor_names:
          tensor_dict[key] = tf.get_default_graph().get_tensor_by_name(
              tensor_name)
      if 'detection_masks' in tensor_dict:
        # The following processing is only for single image
        detection_boxes = tf.squeeze(tensor_dict['detection_boxes'], [0])
        detection_masks = tf.squeeze(tensor_dict['detection_masks'], [0])
        # Reframe is required to translate mask from box coordinates to image coordinates and fit the image size.
        real_num_detection = tf.cast(tensor_dict['num_detections'][0], tf.int32)
        detection_boxes = tf.slice(detection_boxes, [0, 0], [real_num_detection, -1])
        detection_masks = tf.slice(detection_masks, [0, 0, 0], [real_num_detection, -1, -1])
        detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
            detection_masks, detection_boxes, image.shape[0], image.shape[1])
        detection_masks_reframed = tf.cast(
            tf.greater(detection_masks_reframed, 0.5), tf.uint8)
        # Follow the convention by adding back the batch dimension
        tensor_dict['detection_masks'] = tf.expand_dims(
            detection_masks_reframed, 0)
      image_tensor = tf.get_default_graph().get_tensor_by_name('image_tensor:0')

      # Run inference
      output_dict = sess.run(tensor_dict,
                             feed_dict={image_tensor: np.expand_dims(image, 0)})

      # all outputs are float32 numpy arrays, so convert types as appropriate
      output_dict['num_detections'] = int(output_dict['num_detections'][0])
      output_dict['detection_classes'] = output_dict[
          'detection_classes'][0].astype(np.uint8)
      output_dict['detection_boxes'] = output_dict['detection_boxes'][0]
      output_dict['detection_scores'] = output_dict['detection_scores'][0]
      if 'detection_masks' in output_dict:
        output_dict['detection_masks'] = output_dict['detection_masks'][0]
  return output_dict

In [ ]:
import csv

IMAGE_SIZE = (12, 8)
date_c=1
with open('cantidad.csv', mode='w') as csv_file:
    fieldnames = ['date', 'image_name','apple', 'pear', 'lemon','orange','banana','pineapple']
    writer = csv.DictWriter(csv_file, fieldnames=fieldnames)

    writer.writeheader()
    for image_path in TEST_IMAGE_PATHS:
      file_name=os.path.basename(image_path)
      print(file_name)
      image = Image.open(image_path)
      # the array based representation of the image will be used later in order to prepare the
      # result image with boxes and labels on it.
      image_np = load_image_into_numpy_array(image)
      # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
      image_np_expanded = np.expand_dims(image_np, axis=0)
      # Actual detection.
      output_dict = run_inference_for_single_image(image_np, detection_graph)
      vis_util.visualize_boxes_and_labels_on_image_array(
          image_np,
          output_dict['detection_boxes'],
          output_dict['detection_classes'],
          output_dict['detection_scores'],
          category_index,
          instance_masks=output_dict.get('detection_masks'),
          use_normalized_coordinates=True,
          line_thickness=8,
          min_score_thresh=.4)
      
      save_in='drive/My Drive/TestRes/'+file_name
      Image.fromarray(image_np).save(save_in)

      boxes = output_dict['detection_boxes']
      max_boxes_to_draw = boxes.shape[0]
      scores = output_dict['detection_scores']
      classes = output_dict['detection_classes']
      masks = output_dict['detection_masks']
      min_score_thresh=.4
      fruit_counter=[0,0,0,0,0,0]
      fruit_counter ={'apple':0,'pear':0,'lemon':0,'orange':0,'banana':0,'pineapple':0}
      for i in range(min(max_boxes_to_draw, boxes.shape[0])):
          if scores is None or scores[i] > min_score_thresh:
              # boxes[i] is the box which will be drawn
              cat=category_index.get(classes[i])
              print ("This box is gonna get used", boxes[i], float("{:.2f}".format(scores[i]*100)), cat['name'])
              fruit_counter[cat['name']]+=1

      print("Total fruit:", fruit_counter)
      fruit_counter['date']=date_c
      fruit_counter['image_name']=file_name
      date_c+=1
      writer.writerow(fruit_counter)

## Validación de la detección de fruta en las imágenes de producción

In [29]:
import csv
fruits=['apple', 'pear', 'lemon', 'banana', 'pineapple', 'orange']

import pandas as pd 
import pandas_ml as pdml
from pandas_ml import ConfusionMatrix

# Read data from file 'cantidad.csv' 
data = pd.read_csv("cantidad.csv", sep=';') 

#Pasamos los datos numéricos a integer, para poder operar con ellos.
for i in fruits:
    data[i]=data[i].astype('int32')

# Preview the first 5 lines of the loaded data 
print(data.head())

# Read data from file 'cantidad_real.csv' 
data_real = pd.read_csv("cantidad_real.csv", sep=';') 

#Pasamos los datos numéricos a integer, para poder operar con ellos.
for i in fruits:
    data_real[i]=data_real[i].astype('int32')

# Preview the first 5 lines of the loaded data 
data_real.head()

data.head()

date                 image_name  apple  pear  lemon  orange  banana  \
0  20/02/2020    IMG_20200518_165912.jpg      1     3      1       0       3   
1  21/02/2020  IMG_20200518_165931_1.jpg      2     3      3       1       0   
2  22/02/2020    IMG_20200518_165844.jpg      2     3      1       0       3   
3  23/02/2020    IMG_20200518_165857.jpg      2     3      2       1       4   
4  24/02/2020    IMG_20200518_165839.jpg      2     2      1       0       3   

   pineapple  
0          0  
1          0  
2          0  
3          0  
4          0  


,date,image_name,apple,pear,lemon,orange,banana,pineapple
0,20/02/2020,IMG_20200518_165912.jpg,1,3,1,0,3,0
1,21/02/2020,IMG_20200518_165931_1.jpg,2,3,3,1,0,0
2,22/02/2020,IMG_20200518_165844.jpg,2,3,1,0,3,0
3,23/02/2020,IMG_20200518_165857.jpg,2,3,2,1,4,0
4,24/02/2020,IMG_20200518_165839.jpg,2,2,1,0,3,0


In [30]:

for i in fruits[:3]:
    matriz_confusion = ConfusionMatrix(data[i],data_real[i])
    print("Para la fruta ", i, " la exactitud total es del ",matriz_confusion.stats()['overall']['Accuracy']*100, "%")
    print(matriz_confusion.print_stats())

Para la fruta  apple  la exactitud total es del  67.3076923076923 %
Confusion Matrix:

Predicted  0   1   2   3  __all__
Actual                           
0          8   7   2   0       17
1          0  30  10   2       42
2          0   0  27  13       40
3          0   0   0   5        5
__all__    8  37  39  20      104


Overall Statistics:

Accuracy: 0.6730769230769231
95% CI: (0.5741392800895302, 0.7618667125500935)
No Information Rate: ToDo
P-Value [Acc > NIR]: 7.089027107052404e-10
Kappa: 0.5263862844896866
Mcnemar's Test P-Value: ToDo


Class Statistics:

Classes                                       0         1         2          3
Population                                  104       104       104        104
P: Condition positive                        17        42        40          5
N: Condition negative                        87        62        64         99
Test outcome positive                         8        37        39         20
Test outcome negative             

In [31]:

for i in fruits[3:]:
    matriz_confusion = ConfusionMatrix(data[i],data_real[i])
    print("Para la fruta ", i, " la exactitud total es del ",matriz_confusion.stats()['overall']['Accuracy']*100, "%")
    print(matriz_confusion.print_stats())

Para la fruta  banana  la exactitud total es del  67.3076923076923 %
Confusion Matrix:

Predicted  0   1   2   3   4  __all__
Actual                               
0          6   1   0   0   0        7
1          0  19  13   3   0       35
2          0   1  27   5   0       33
3          0   0   2  15   9       26
4          0   0   0   0   3        3
__all__    6  21  42  23  12      104


Overall Statistics:

Accuracy: 0.6730769230769231
95% CI: (0.5741392800895302, 0.7618667125500935)
No Information Rate: ToDo
P-Value [Acc > NIR]: 2.678170710691295e-08
Kappa: 0.5590472627509665
Mcnemar's Test P-Value: ToDo


Class Statistics:

Classes                                        0          1          2  \
Population                                   104        104        104   
P: Condition positive                          7         35         33   
N: Condition negative                         97         69         71   
Test outcome positive                          6         21       

## Tratamiento de los datos para la predicción de compra

In [32]:
# Read data from file 'cantidad.csv' 
data = pd.read_csv("cantidad.csv", sep=';') 
# Preview the first 5 lines of the loaded data 
data.head()

,date,image_name,apple,pear,lemon,orange,banana,pineapple
0,20/02/2020,IMG_20200518_165912.jpg,1,3,1,0,3,0
1,21/02/2020,IMG_20200518_165931_1.jpg,2,3,3,1,0,0
2,22/02/2020,IMG_20200518_165844.jpg,2,3,1,0,3,0
3,23/02/2020,IMG_20200518_165857.jpg,2,3,2,1,4,0
4,24/02/2020,IMG_20200518_165839.jpg,2,2,1,0,3,0


Pasamos los datos numéricos a integer, para poder operar con ellos. 

In [33]:
for i in fruits:
    data[i]=data[i].astype('int32')

Generamos la tabla de consumo, calculando la diferencia entre un dia y su anterior. 

In [34]:
data_diff=data.copy()
for i in fruits:
    data_diff[i]=data_diff[i].diff()
data_diff.head()

,date,image_name,apple,pear,lemon,orange,banana,pineapple
0,20/02/2020,IMG_20200518_165912.jpg,NaN,NaN,NaN,NaN,NaN,NaN
1,21/02/2020,IMG_20200518_165931_1.jpg,1.0,0.0,2.0,1.0,-3.0,0.0
2,22/02/2020,IMG_20200518_165844.jpg,0.0,0.0,-2.0,-1.0,3.0,0.0
3,23/02/2020,IMG_20200518_165857.jpg,0.0,0.0,1.0,1.0,1.0,0.0
4,24/02/2020,IMG_20200518_165839.jpg,0.0,-1.0,-1.0,-1.0,-1.0,0.0


Generamos una tabla de compras, para crear un histórico, en la que añadiremos todos los incrementos de fruta detectados, es decir filas con valores positivos, que podrían deberse a compras realizadas.

In [35]:
data_pos=data_diff.copy()
for i in fruits:
    data_pos.loc[data_pos[i] < 0, i] = 0

data_pos.head()

,date,image_name,apple,pear,lemon,orange,banana,pineapple
0,20/02/2020,IMG_20200518_165912.jpg,NaN,NaN,NaN,NaN,NaN,NaN
1,21/02/2020,IMG_20200518_165931_1.jpg,1.0,0.0,2.0,1.0,0.0,0.0
2,22/02/2020,IMG_20200518_165844.jpg,0.0,0.0,0.0,0.0,3.0,0.0
3,23/02/2020,IMG_20200518_165857.jpg,0.0,0.0,1.0,1.0,1.0,0.0
4,24/02/2020,IMG_20200518_165839.jpg,0.0,0.0,0.0,0.0,0.0,0.0


De la tabla de compras generada, eliminamos las filas en las que no se ha comprado nada, no ha habido compra. 

In [36]:
rows_zeros=data_pos[(data_pos['apple']==0) & (data_pos['lemon']==0) & (data_pos['orange']==0) & (data_pos['pear']==0) & (data_pos['pineapple']==0) & (data_pos['banana']==0)].index

data_pos.drop(rows_zeros , inplace=True)
data_pos.head()

,date,image_name,apple,pear,lemon,orange,banana,pineapple
0,20/02/2020,IMG_20200518_165912.jpg,NaN,NaN,NaN,NaN,NaN,NaN
1,21/02/2020,IMG_20200518_165931_1.jpg,1.0,0.0,2.0,1.0,0.0,0.0
2,22/02/2020,IMG_20200518_165844.jpg,0.0,0.0,0.0,0.0,3.0,0.0
3,23/02/2020,IMG_20200518_165857.jpg,0.0,0.0,1.0,1.0,1.0,0.0
6,26/02/2020,IMG_20200518_165828.jpg,0.0,0.0,2.0,0.0,0.0,0.0


Procedemos a restar las piezas compradas de la tabla de consumos, de esta forma solo deberiamos ver valores iguales o inferiores a 0, en caso de detectar un valor por encima de 0 implicaria un fallo, puesto que si no se ha comprado fruta no puede haber más. Se reportará como un error y se sustituirá por un 0.

In [37]:

for i in fruits:
    data_diff[i]=data_diff[i].sub(data_pos[i])
data_diff.head()

,date,image_name,apple,pear,lemon,orange,banana,pineapple
0,20/02/2020,IMG_20200518_165912.jpg,NaN,NaN,NaN,NaN,NaN,NaN
1,21/02/2020,IMG_20200518_165931_1.jpg,0.0,0.0,0.0,0.0,-3.0,0.0
2,22/02/2020,IMG_20200518_165844.jpg,0.0,0.0,-2.0,-1.0,0.0,0.0
3,23/02/2020,IMG_20200518_165857.jpg,0.0,0.0,0.0,0.0,0.0,0.0
4,24/02/2020,IMG_20200518_165839.jpg,NaN,NaN,NaN,NaN,NaN,NaN


In [38]:
tmp_df=data_diff[(data_diff['apple']>0) | (data_diff['lemon']>0) | (data_diff['orange']>0) | (data_diff['pear']>0) | (data_diff['pineapple']>0) | (data_diff['banana']>0)]

if not tmp_df.empty:
    for i in fruits:
        #Sustituimos todos los valores que han dado error por ser superiores a 0 de la tabla de consumos
        data_diff.loc[data_diff[i] > 0, i] = 0
    print("ERROR: se ha detectado un incremento de la cantidad de fruta, \n sin haber reportado una compra. \n La información erronea es la siguiente: \n",tmp_df)
else:
    print("Datos validados correctamente, no hay ningún incremento de fruta que no se deba a una compra")

Datos validados correctamente, no hay ningún incremento de fruta que no se deba a una compra


In [39]:
data_diff.head()

,date,image_name,apple,pear,lemon,orange,banana,pineapple
0,20/02/2020,IMG_20200518_165912.jpg,NaN,NaN,NaN,NaN,NaN,NaN
1,21/02/2020,IMG_20200518_165931_1.jpg,0.0,0.0,0.0,0.0,-3.0,0.0
2,22/02/2020,IMG_20200518_165844.jpg,0.0,0.0,-2.0,-1.0,0.0,0.0
3,23/02/2020,IMG_20200518_165857.jpg,0.0,0.0,0.0,0.0,0.0,0.0
4,24/02/2020,IMG_20200518_165839.jpg,NaN,NaN,NaN,NaN,NaN,NaN


In [40]:

data_pos.head()

,date,image_name,apple,pear,lemon,orange,banana,pineapple
0,20/02/2020,IMG_20200518_165912.jpg,NaN,NaN,NaN,NaN,NaN,NaN
1,21/02/2020,IMG_20200518_165931_1.jpg,1.0,0.0,2.0,1.0,0.0,0.0
2,22/02/2020,IMG_20200518_165844.jpg,0.0,0.0,0.0,0.0,3.0,0.0
3,23/02/2020,IMG_20200518_165857.jpg,0.0,0.0,1.0,1.0,1.0,0.0
6,26/02/2020,IMG_20200518_165828.jpg,0.0,0.0,2.0,0.0,0.0,0.0


In [41]:

data.head()

,date,image_name,apple,pear,lemon,orange,banana,pineapple
0,20/02/2020,IMG_20200518_165912.jpg,1,3,1,0,3,0
1,21/02/2020,IMG_20200518_165931_1.jpg,2,3,3,1,0,0
2,22/02/2020,IMG_20200518_165844.jpg,2,3,1,0,3,0
3,23/02/2020,IMG_20200518_165857.jpg,2,3,2,1,4,0
4,24/02/2020,IMG_20200518_165839.jpg,2,2,1,0,3,0


## Generamos métricas para la predicción de compra
Calculamos la cantidad de fruta que se compra al día de manera más frecuente, en base al histórico de compras.

In [42]:
number_rows=data_pos.shape[0]
cantidad_fruta={}
for i in fruits:
    s=data_pos[i].value_counts().sort_values(ascending=False)
    
    #print(data_pos[i].value_counts().sort_values(ascending=False))
    j=s.index[s.index>0].values
    number_of_fruit_bought=0
    for elem in range(0, len(j)):
        number_of_fruit_bought+=s[j[elem]]
    #print(s)
    print("La clase de fruta "+i+" tiene como cantidad de fruta comprada mas frecuentemente: ", j[0], " con el numero de occurencias = ",s[j[0]], " y la cantidad total comprada = ", number_of_fruit_bought)
    cantidad_fruta[i]=j[0]

La clase de fruta apple tiene como cantidad de fruta comprada mas frecuentemente:  1.0  con el numero de occurencias =  18  y la cantidad total comprada =  20
La clase de fruta pear tiene como cantidad de fruta comprada mas frecuentemente:  1.0  con el numero de occurencias =  13  y la cantidad total comprada =  16
La clase de fruta lemon tiene como cantidad de fruta comprada mas frecuentemente:  1.0  con el numero de occurencias =  18  y la cantidad total comprada =  22
La clase de fruta banana tiene como cantidad de fruta comprada mas frecuentemente:  1.0  con el numero de occurencias =  13  y la cantidad total comprada =  18
La clase de fruta pineapple tiene como cantidad de fruta comprada mas frecuentemente:  1.0  con el numero de occurencias =  4  y la cantidad total comprada =  7
La clase de fruta orange tiene como cantidad de fruta comprada mas frecuentemente:  1.0  con el numero de occurencias =  19  y la cantidad total comprada =  22


Calculamos el consumo medio diario por tipo de fruta y el consumo diario medio de fruta.

In [43]:
data_diff["fruta"] = data_diff[fruits].sum(axis=1)

In [44]:

import math
media_consumo_fruta={}
s=data_diff.mean()
#print(data_diff.mean())
for i,j in zip(data_diff._get_numeric_data().columns,range(0,len(s))):
    if(i!="date"):
        print("La media de ", i, " consumida por dia es: ",math.ceil(abs(s[j])))
        media_consumo_fruta[i]=math.ceil(abs(s[j]))

La media de  apple  consumida por dia es:  1
La media de  pear  consumida por dia es:  1
La media de  lemon  consumida por dia es:  1
La media de  orange  consumida por dia es:  1
La media de  banana  consumida por dia es:  1
La media de  pineapple  consumida por dia es:  1
La media de  fruta  consumida por dia es:  1


Calculamos la cantidad de piezas de fruta, por tipo, que hay en la nevera diariamente.

In [45]:
data["fruta"] = data[fruits].sum(axis=1)
s=data.mean()
media_cantidad_fruta={}
#print(data_diff.mean())
for i,j in zip(data._get_numeric_data().columns,range(0,len(s))):
    if(i!="date"):
        print("La media de ", i, " por dia en la nevera: ",math.ceil(abs(s[j])))
        media_cantidad_fruta[i]=math.ceil(abs(s[j]))

La media de  apple  por dia en la nevera:  2
La media de  pear  por dia en la nevera:  1
La media de  lemon  por dia en la nevera:  1
La media de  orange  por dia en la nevera:  1
La media de  banana  por dia en la nevera:  2
La media de  pineapple  por dia en la nevera:  1
La media de  fruta  por dia en la nevera:  6


Calculamos la media de dias que pasan entre compra y compra.

In [46]:
import datetime

dates=pd.to_datetime(data_pos['date'])
diff_dates=dates.diff()

dates.head()
dias_mean=diff_dates.mean()
print("La media de dias que pasan entre una compra y la siguiente es: ", diff_dates.mean())



La media de dias que pasan entre una compra y la siguiente es:  -1 days +09:12:19.726027


## Predicción de qué se debe comprar
Calculamos la cantidad de fruta que se debe comprar, dada la cantidad de fruta que se observa en la nevera, la que suele haber, la que se suele consumir y cada cuanto se suele ir a comprar.

In [47]:
valor_actual_nevera=[0,3,2,1,2,0]
final_dict_nevera={}
for i,j in zip(fruits, valor_actual_nevera):
    final_dict_nevera[i]=j
import re
final_dict={}
for i,j in zip(fruits, valor_actual_nevera):
    tardo_en_comprar=re.sub("days.*",'',str(dias_mean))
    cantidad_fruta=int(tardo_en_comprar)*media_consumo_fruta[i]
    print("Teniendo en cuenta que se compra cada ", tardo_en_comprar, " dias y que por tanto si el consumo diario es de ",media_consumo_fruta[i], ", consumiré ", cantidad_fruta, i)
    print("Que actualmente hay ", j, i, " en la nevera.")
    print("Que lo habitual es tener en la nevera ", media_cantidad_fruta[i], i )
    res=media_cantidad_fruta[i]-j
    if res > 0: 
        print("Debo comprar: ", max(res, cantidad_fruta-j))
        final_dict[i]=max(res, cantidad_fruta-j)
    else:
        if(cantidad_fruta-j<=0):
                print("Debo comprar: ", 0)
                final_dict[i]=0
        else:
            final=cantidad_fruta+res
            print("Debo comprar: ", final)
            final_dict[i]=final
    

print("\n\n##################################")
print("\nEn la nevera hay: ")
print(final_dict_nevera)
print("\n##################################")


print("\n\n##################################")
print("\nLa lista de la compra final es: ")
print(final_dict)
print("\n##################################")




Teniendo en cuenta que se compra cada  -1   dias y que por tanto si el consumo diario es de  1 , consumiré  -1 apple
Que actualmente hay  0 apple  en la nevera.
Que lo habitual es tener en la nevera  2 apple
Debo comprar:  2
Teniendo en cuenta que se compra cada  -1   dias y que por tanto si el consumo diario es de  1 , consumiré  -1 pear
Que actualmente hay  3 pear  en la nevera.
Que lo habitual es tener en la nevera  1 pear
Debo comprar:  0
Teniendo en cuenta que se compra cada  -1   dias y que por tanto si el consumo diario es de  1 , consumiré  -1 lemon
Que actualmente hay  2 lemon  en la nevera.
Que lo habitual es tener en la nevera  1 lemon
Debo comprar:  0
Teniendo en cuenta que se compra cada  -1   dias y que por tanto si el consumo diario es de  1 , consumiré  -1 banana
Que actualmente hay  1 banana  en la nevera.
Que lo habitual es tener en la nevera  2 banana
Debo comprar:  1
Teniendo en cuenta que se compra cada  -1   dias y que por tanto si el consumo diario es de  1 , con